In [110]:
import os
from os.path import expanduser, join, basename, dirname
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, mapping
import contextily as ctx
import ulmo
import rasterio as rio
from rasterio.features import dataset_features
from rasterio.mask import mask
from shapely.ops import unary_union
from conversion import imperial_to_metric
from rasterio.windows import Window
from shapely.geometry import mapping
import pickle
from ulmo_extract import get_snotel_image_results
from rio_geom import rio_to_exterior

In [2]:
#This is the latest CUAHSI API endpoint
wsdlurl = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'
sites = ulmo.cuahsi.wof.get_sites(wsdlurl)

In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print('Starting snotel sd vs uavsar sd analysis...')
res = pd.DataFrame()

with open(expanduser('~/scratch/data/uavsar/image_fps'), 'rb') as f:
    image_fps = pickle.load(f)

image_fps = [f for f in image_fps if f['fp'].endswith('.unw.grd.tiff')]
image_fps = iter(image_fps)

tmp_dir = expanduser(f'~/uavsar/results/uavsar_snotel_sd/tmp/')

def process(img):
    dic = {}
    desc = pd.read_csv(img['ann'], index_col = 0)
    dic['name'] = basename(img['fp'])
    dic['first_dt'] = pd.to_datetime(desc.loc['value', 'start time of acquisition for pass 1'])
    dic['second_dt'] = pd.to_datetime(desc.loc['value', 'start time of acquisition for pass 2'])
    dic['pol'] = basename(img['fp']).split('_')[6][4:]
    dic['snotel_result'] = get_snotel_image_results(img['fp'], img['inc'], img['ann'])
    dic['snow_off_phase'] = 
    # res = pd.concat([res, pd.DataFrame.from_records([dic])])
    # with open(join(tmp_dir, basename(img['fp'])), 'wb') as f:
    #     pickle.dump(dic, f)
    return dic

Starting snotel sd vs uavsar sd analysis...


In [7]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

In [9]:
# This function computes the feature's geometry area and adds it as a property.
def addArea(feature):
  return feature.set({'areaHa': feature.geometry().area(10).divide(100 * 100)})

In [109]:
def snow_off_phase(img_fp, ann_fp):
    geo = rio_to_exterior(img_fp, simplify= True)
    with rio.open(img_fp) as src:
        bounds = src.bounds
        geom = ee.Geometry.BBox(*bounds)
        df = pd.read_csv(ann_fp, index_col = [0])
        s = pd.to_datetime(df.loc['value','start time of acquisition for pass 1']).tz_localize(None)
        e = pd.to_datetime(df.loc['value','start time of acquisition for pass 2']).tz_localize(None)
        try:
            snow_cover = ee.ImageCollection("MODIS/006/MOD10A1").select('NDSI_Snow_Cover').filterDate(s,e).mean().clip(geom)
            vectors = snow_cover.gt(10).reduceToVectors(scale=500).filterMetadata('label','equals',0)
            # Map the area getting function over the FeatureCollection.
            areaAdded = vectors.map(addArea)
            areaAdded = areaAdded.filterMetadata('areaHa','greater_than',10)
            if areaAdded.size().getInfo()>0:
                f = areaAdded.limit(1, 'areaHa', False)
                p = Polygon(f.first().geometry().coordinates().getInfo()[0])
                geoms = [mapping(p)]
                phase_sub, _ = mask(src, geoms, crop=True)
                if np.count_nonzero(~np.isnan(phase_sub)) > 0:
                    snow_off_phase = np.nanmean(phase_sub[0])
                    return snow_off_phase
        except ee.ee_exception.EEException as e:
            print(e)

In [83]:
with open(expanduser('~/scratch/data/uavsar/image_fps'), 'rb') as f:
    image_fps = pickle.load(f)

image_fps = [f for f in image_fps if f['fp'].endswith('.unw.grd.tiff')]

In [97]:
def snow_off_phase(img_fp, ann_fp):
    j = 3
    with rio.open(img_fp) as src:
        bounds = src.bounds
        geom = ee.Geometry.BBox(*bounds)
        df = pd.read_csv(ann_fp, index_col = [0])
        s = pd.to_datetime(df.loc['value','start time of acquisition for pass 1']).tz_localize(None)
        e = pd.to_datetime(df.loc['value','start time of acquisition for pass 2']).tz_localize(None)
        try:
            snow_cover = ee.ImageCollection("MODIS/006/MOD10A1").select('NDSI_Snow_Cover').filterDate(s,e).mean().clip(geom)
            vectors = snow_cover.gt(10).reduceToVectors(scale = 30, maxPixels = 1e9, bestEffort= True).filterMetadata('label','equals',0)
            # Map the area getting function over the FeatureCollection.
            vectors = vectors.map(addArea)
            f = vectors.filterMetadata('areaHa','greater_than',5)
            if vectors.size().getInfo()>0:
                f = f.toList(j)
                for i in range(j):
                    p = Polygon(f.get(i).getInfo()['geometry']['coordinates'][0])
                    geoms = [mapping(p)]
                    phase_sub, _ = mask(src, geoms, crop=True)
                    if np.count_nonzero(~np.isnan(phase_sub)) > 0:
                        snow_off_phase = np.nanmean(phase_sub[0])
                        return snow_off_phase
        except ee.ee_exception.EEException as e:
            print(e)

In [105]:
def snow_off_phase_check(img_fp, ann_fp):
    j = 1
    with rio.open(img_fp) as src:
        bounds = src.bounds
        geom = ee.Geometry.BBox(*bounds)
        df = pd.read_csv(ann_fp, index_col = [0])
        s = pd.to_datetime(df.loc['value','start time of acquisition for pass 1']).tz_localize(None)
        e = pd.to_datetime(df.loc['value','start time of acquisition for pass 2']).tz_localize(None)
        try:
            snow_cover = ee.ImageCollection("MODIS/006/MOD10A1").select('NDSI_Snow_Cover').filterDate(s,e).mean().clip(geom)
            vectors = snow_cover.gt(10).reduceToVectors(scale = 30, maxPixels = 1e9, bestEffort= True).filterMetadata('label','equals',0)
            # Map the area getting function over the FeatureCollection.
            vectors = vectors.map(addArea)
            f = vectors.filterMetadata('areaHa','greater_than',5)
            if vectors.size().getInfo()>0:
                f1 = f.toList(f.size().getInfo())
                f1 = f1.sort()
                p1 = Polygon(f1.get(0).getInfo()['geometry']['coordinates'][0])

                f2 = f.limit(1, 'areaHa', False)
                p2 = Polygon(f2.first().geometry().coordinates().getInfo()[0])
                print(p1)
                print(p2)
                print(p1 == p2)

                
        except ee.ee_exception.EEException as e:
            print(e)

In [106]:
image_fps = [i for i in image_fps if 'L090VH_' in i['fp']]
for i in range(40):
    snow_off_phase_check(image_fps[i]['fp'],image_fps[i]['ann'])

List.sort: Cannot compare values '<Feature>' (Type<Feature>) and '<Feature>' (Type<Feature>).
List.sort: Cannot compare values '<Feature>' (Type<Feature>) and '<Feature>' (Type<Feature>).


KeyboardInterrupt: 

In [108]:
image_fps = [i for i in image_fps if 'L090VH_' in i['fp']]
for i in range(40):
    print(snow_off_phase(image_fps[i]['fp'],image_fps[i]['ann']))

2.5160327
-0.4435453
0.54857254
-0.74427754


KeyboardInterrupt: 